In [1]:
LAB_USER_NAME = 'DEMO'

**Important**: To initialise this notebook, edit the cell above to set `LAB_USER_NAME` to your name, then click **Run->Run All Cells** in the top menu bar. This needs to be the same as the name used in **Part 1** for the files from that part to be loaded.

In [2]:
import panel as pn
pn.extension()
import sys
import os
sys.path.append('/home/notebooks/dashboards-inline')
LAB_DIR = os.path.join('/home/data/', LAB_USER_NAME)
os.makedirs(LAB_DIR, exist_ok=True)
print('User data directory:', LAB_DIR)

User data directory: /home/data/DEMO


## Part Four: Single Shot T2 Measurement

### 1. Overview



### 2. Theory

Spin-spin relaxation causes the NMR signal to decay exponentially via dephasing with a time constant called $T_2$, described by the following equation:

$$\large M(t) = M(0) \cdot \mathrm{e}^{-t/T_2}$$

### 3. The Carr-Purcell Meiboom-Gill Sequence
The dashboard below allows you to modify the echo time and echo count and see the NMR signal resulting from a CPMG (Carr-Purcell Meiboom-Gill) pulse train. For this pulse sequence the signal is acquired the entire time from the first RF pulse, so you will be able to see the FID signal and glitches caused by the RF  pulse train, as well as the signal from echoes. Usually the acquisition would be gated on for a short time around the center of each echo.

#### **Phase Cycling**
This tool allows for multiple scans to be acquired. When using multiple scans the phases of the 90 and 180 pulses will be cycled according to the following scheme:
| Scan | 90 | 180 |
| -: | -: | -: |
| 1 | 0 | 90 |
| 2 | 180 | 90 |
| 3 | 0 | 270 |
| 4 | 180 | 270 |

#### **Plot Tool Notes**
The $y$ value can be easily read off a plot line by hovering over it with the mouse, as long as the hover tool is enabled.

#### **Step One**
Run the CPMG experiment. You should be able to see the $T_2$ relaxation in the heights of the echos

In [32]:
# inputs
s3_echo_time = pn.widgets.FloatInput(name='Echo Time (seconds)', start=0.005, end=0.1, step=0.001, value=0.02, width=200)
s3_echo_count = pn.widgets.IntInput(name='Echo Count', start=1, end=64, step=1, value=16, width=200)
s3_scan_count = pn.widgets.IntInput(name='Scans (phase cycled)', start=1, end=4, step=1, value=1, width=200)
    
from full_acq_CPMG import FullAcqCPMGApp # from dashboards-inline directory that was added to sys.path
# set some parameters directly
override_pars = dict(
    t_echo=s3_echo_time,
    n_echo=s3_echo_count,
    n_scans=s3_scan_count,
    n_samples=25000,
    t_dw=20e-6,
    t_end=0.5
)

# create dashboard app
section3_app = FullAcqCPMGApp(
    override_pars=override_pars,
    show_magnitude=True,
    show_complex=True
)

# display layout
pn.Column(
    pn.Row(s3_echo_time, s3_echo_count, s3_scan_count),
    section3_app.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] Row
        [0] FloatInput(end=0.1, name='Echo Time (seconds)', start=0.005, step=0.001, value=0.02, width=200)
        [1] IntInput(end=64, name='Echo Count', start=1, value=16, width=200)
        [2] IntInput(end=4, name='Scans (phase cycled)', start=1, value=1, width=200)
    [1] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [2] Progress(height=31, sizing_mode='stretch_width', value=0)
            [3] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

### 4. CPMG with short echo time and many echos

This experiment only acquires signal in a series of short windows centered on the echos, then the signal from each echo is averaged into a single data point.
With the data processed in this way the result is a direct plot of the decay curve of the heights of the echos.

**Note:** The echo time, echo count, and scan count inputs here are not syncronized to the ones above.

In [33]:
from CPMG import CPMGApp # from dashboards-inline directory that was added to sys.path

s4_echo_time = pn.widgets.FloatInput(name='Echo Time (seconds)', start=0.0002, step=0.0001, end=0.01, value=0.001, width=200)
s4_echo_count = pn.widgets.IntInput(name='Echo Count', start=1, end=10000, step=1, value=2000, width=200)
s4_scan_count = pn.widgets.IntInput(name='Scans (phase cycled)', start=1, end=4, step=1, value=1, width=200)

# set some parameters directly
override_pars = dict(
    t_echo=s4_echo_time,
    n_echo=s4_echo_count,
    n_scans=s4_scan_count,
    n_samples=64,
    t_dw=1e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_end=0.5
)

# create dashboard app
section4_app = CPMGApp(
    override_pars=override_pars,
    enable_run_loop=True,
    show_magnitude=True
)

# display layout
pn.Column(
    pn.Row(s4_echo_time, s4_echo_count, s4_scan_count),
    section4_app.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] Row
        [0] FloatInput(end=0.01, name='Echo Time (seconds)', start=0.0002, step=0.0001, value=0.001, width=200)
        [1] IntInput(end=10000, name='Echo Count', start=1, value=2000, width=200)
        [2] IntInput(end=4, name='Scans (phase cycled)', start=1, value=1, width=200)
    [1] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='success', name='Run Loop', sizing_mode='stretch_width')
            [2] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [3] Progress(height=31, sizing_mode='stretch_width', value=0)
            [4] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')